In [1]:
import cv2
import numpy as np

In [12]:
image = cv2.imread('light6.jpg') # 根据路径读取一张图片
image = cv2.GaussianBlur(image, (7,7), 0)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # BGR转HSV
# 绿色的范围
lower_green = np.array([35, 20, 147])
upper_green = np.array([70, 255, 255])

# 红色的范围
low_red = np.array([0, 56, 90])
high_red = np.array([9, 203, 255])

#黄色的范围
low_yellow = np.array([16, 20, 200])
high_yellow = np.array([36, 235, 255])

# 3.inRange()：介于lower/upper之间的为白色，其余黑色
 #将低于阈值和高于阈值的部分变成0，中间部分变为255
mask = cv2.inRange(hsv, low_red, high_red)
# 4.只保留原图中的特定阈值mask ≠0时img与img按位相与，即显示特定颜色部分，其他部分为0（黑色）
res = cv2.bitwise_and(image, image, mask=mask)
# cv2.imshow('frame', image)
# cv2.imshow('mask', mask)
# cv2.imwrite('mask.jpg',mask)
# cv2.imshow('res', res)

#降噪（模糊处理用来减少瑕疵点）
blur = cv2.blur(res, (5,5))

#灰度化
gray=cv2.cvtColor(blur,cv2.COLOR_BGR2GRAY)

#形态学运算，膨胀  看一下在前面膨胀还是后面膨胀效果更好一点
# dilated = cv2.dilate(gray, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=2)

# 其中较大的阈值2用于检测图像中明显的边缘，但一般情况下检测的效果不会那么完美，边缘检测出来是断断续续的。
# 所以这时候用较小的第一个阈值用于将这些间断的边缘连接起来
canny = cv2.Canny(gray, 150, 240) 
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (100, 100))
closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

cv2.imshow('gray', gray)
cv2.imshow('canny', canny)
cv2.imshow('closed', closed)
cv2.imwrite('closed.jpg',closed)

#霍夫变换圆检测
circles= cv2.HoughCircles(canny,cv2.HOUGH_GRADIENT,1,100,param1=100,param2=70,minRadius=0,maxRadius=130)
# 先排个序,确定最上面的是红灯，中间的是黄灯，最后的是绿灯
print(type(circles))

cv2.waitKey(0)
cv2.destroyAllWindows()


<class 'NoneType'>


In [38]:
img = cv2.imread('mask.jpg')
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)
image ,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#绘制独立轮廓，如第四个轮廓
#imag = cv2.drawContour(img,contours,-1,(0,255,0),3)
#但是大多数时候，下面方法更有用
imag = cv2.drawContours(img,contours,20,(0,255,0),3)

In [39]:
while(1):
    cv2.imshow('img',img)
    cv2.imshow('imgray',imgray)
    cv2.imshow('image',image)
    cv2.imshow('imag',imag)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

In [7]:
import numpy as np
import cv2

img = cv2.imread('closed.jpg',0)

ret,thresh = cv2.threshold(img,127,255,0)
image,contours,hierarchy=cv2.findContours(thresh,1,2)
cnt=contours[0]
# print(cnt)
# M=cv2.moments(cnt)
# print(len(M),M.keys())

# 轮廓面积
area=cv2.contourArea(cnt)
# print(area)

# 轮廓周长
perimeter = cv2.arcLength(cnt,True)

# 轮廓近似
epsilon=0.1*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)

# hull = cv2.convexHull(points,hull,clockwise,returnPoints)
hull=cv2.convexHull(cnt,returnPoints=True)

k=cv2.isContourConvex(cnt)
# print(k)

# 最小面积边界矩形
# x,y,w,h=cv2.boundingRect(cnt)
# print(x,y,w,h)
# img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

# 最小外接圆
# 函数cv2.minEnclosingCircle()可以帮我们找到一个对象的外接圆。它是所有能够包括对象的圆中面积最小的一个。

(x,y),radius = cv2.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
img = cv2.circle(img,center,radius,(0,0,255),2)

# # 椭圆拟合
# # 使用函数cv2.ellipse()，返回值其实就是旋转边界矩形的内切圆。

# ellipse = cv2.fitEllipse(cnt)
# img = cv2.ellipse(img,ellipse,(0,255,0),2)
# print(img.shape)
res=img[int(y-radius):int(y+radius),int(x-radius):int(x+radius)]#已经剪切出目标区域
cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
import cv2
history = 20    # 训练帧数

bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)  # 背景减除器，设置阴影检测
bs.setHistory(history)

frames = 0

while True:
#         res, frame = camera.read()
    frame=cv2.imread('test0.jpg')
#         if not res:
#             break

    fg_mask = bs.apply(frame)   # 获取 foreground mask

    if frames < history:
        frames += 1
        continue

    # 对原始帧进行膨胀去噪
    th = cv2.threshold(fg_mask.copy(), 244, 255, cv2.THRESH_BINARY)[1]
    th = cv2.erode(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=2)
    dilated = cv2.dilate(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 3)), iterations=2)
    # 获取所有检测框
    image, contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # 获取矩形框边界坐标
        x, y, w, h = cv2.boundingRect(c)
        # 计算矩形框的面积
        area = cv2.contourArea(c)
        if 500 < area < 3000:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("detection", frame)
    cv2.imshow("back", dilated)
    if cv2.waitKey(110) & 0xff == 27:
        break
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

#加载原图
img=cv2.imread('light.jpg')
print('img:',type(img),img.shape,img.dtype)
cv2.imshow('img',img)

hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
cv2.imshow('hsv',hsv)

#提取蓝色区域
blue_lower=np.array([100,50,50])
blue_upper=np.array([124,255,255])
mask=cv2.inRange(hsv,blue_lower,blue_upper)
print('mask',type(mask),mask.shape)
cv2.imshow('mask',mask)

#模糊
blurred=cv2.blur(mask,(9,9))
cv2.imshow('blurred',blurred)
#二值化
ret,binary=cv2.threshold(blurred,127,255,cv2.THRESH_BINARY)
cv2.imshow('blurred binary',binary)

#使区域闭合无空隙
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
cv2.imshow('closed',closed)

#腐蚀和膨胀
'''
腐蚀操作将会腐蚀图像中白色像素，以此来消除小斑点，
而膨胀操作将使剩余的白色像素扩张并重新增长回去。
'''
erode=cv2.erode(closed,None,iterations=4)
cv2.imshow('erode',erode)
dilate=cv2.dilate(erode,None,iterations=4)
cv2.imshow('dilate',dilate)

# 查找轮廓
image,contours, hierarchy=cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
print('轮廓个数：',len(contours))
i=0
res=img.copy()
for con in contours:
    #轮廓转换为矩形
    rect=cv2.minAreaRect(con)
    #矩形转换为box
    box=np.int0(cv2.boxPoints(rect))
    #在原图画出目标区域
    cv2.drawContours(res,[box],-1,(0,0,255),2)
#     print([box])
    #计算矩形的行列
    h1=max([box][0][0][1],[box][0][1][1],[box][0][2][1],[box][0][3][1])
    h2=min([box][0][0][1],[box][0][1][1],[box][0][2][1],[box][0][3][1])
    l1=max([box][0][0][0],[box][0][1][0],[box][0][2][0],[box][0][3][0])
    l2=min([box][0][0][0],[box][0][1][0],[box][0][2][0],[box][0][3][0])
    print('h1',h1)
    print('h2',h2)
    print('l1',l1)
    print('l2',l2)
    #加上防错处理，确保裁剪区域无异常
    if h1-h2>0 and l1-l2>0:
        #裁剪矩形区域
        temp=img[h2:h1,l2:l1]
        i=i+1
        #显示裁剪后的标志
        cv2.imshow('sign'+str(i),temp)
#显示画了标志的原图       
cv2.imshow('res',res)

cv2.waitKey(0)
cv2.destroyAllWindows()

img: <class 'numpy.ndarray'> (640, 311, 3) uint8
mask <class 'numpy.ndarray'> (640, 311)
轮廓个数： 10
h1 591
h2 528
l1 70
l2 37
h1 491
h2 483
l1 264
l2 256
h1 478
h2 464
l1 54
l2 39
h1 416
h2 400
l1 231
l2 186
h1 445
h2 300
l1 125
l2 36
h1 283
h2 274
l1 262
l2 253
h1 268
h2 226
l1 57
l2 36
h1 238
h2 225
l1 110
l2 97
h1 255
h2 221
l1 238
l2 192
h1 222
h2 214
l1 235
l2 227


In [3]:
img=cv2.imread('light.jpg')
HSV=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
# 蓝色的范围，不同光照条件下不一样，可灵活调整
# 绿色的范围
lower_green = np.array([35, 20, 35])
upper_green = np.array([70, 255, 255])
# 红色的范围
low_red = np.array([50, 80, 90])
high_red = np.array([255, 255, 255])

# 3.inRange()：介于lower/upper之间的为白色，其余黑色
 #将低于lower_blue和高于upper_blue的部分变成0，中间部分变为255（本文中即蓝色部分为255）
# Step3. 形态学运算，膨胀

mask= cv2.inRange(HSV, low_red, high_red)
dilated= cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=2)

# 4.只保留原图中的蓝色部分
 #mask ≠0时frame与frame按位相与，即显示蓝色部分，其他部分为0（黑色）
res = cv2.bitwise_and(img, img, mask=dilated)
cv2.imshow('frame', img)
# cv2.imshow('mask', mask)
cv2.imshow('res', res)
cv2.waitKey(0)  
cv2.destroyAllWindows()

In [4]:

import cv2
import numpy as np

# HSV中
# Blue：[[[120 255 255]]]
# Green：[[[ 60 255 255]]]
# Red：[[[  0 255 255]]]

capture = cv2.VideoCapture(1)

# 蓝色的范围
lower_blue = np.array([100, 110, 110])
upper_blue = np.array([130, 255, 255])

# 绿色的范围
lower_green = np.array([40, 90, 90])
upper_green = np.array([70, 255, 255])

# 红色的范围
lower_red = np.array([160, 120, 120])
upper_red = np.array([179, 255, 255])

while(True):
    # 1.捕获视频中的一帧
    ret, frame = capture.read()

    # 2.从BGR转换到HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # 3.将所有的mask相加，就可以同时显示了
    mask = mask_blue + mask_green + mask_red

    # 4.保留原图中的三种颜色部分
    res = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow('frame', frame)
    cv2.imshow('mask', mask)
    cv2.imshow('res', res)

    if cv2.waitKey(1) == ord('q'):
        break

error: C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:11178: error: (-215) (scn == 3 || scn == 4) && (depth == 0 || depth == 5) in function cv::cvtColor


In [5]:
im=Image.open("light.jpg")
print (im.mode )                      #查看图像的模式
im1=im.convert("YCbCr")     #转换图像的模式到视频模式
y,cb,cr=im1.getpixel((10,10)) #提取点(10,10)位置的亮度、蓝色分量、红色分量的值。 
print (y   )   

NameError: name 'Image' is not defined